In [1]:
import gc
import glob
import os
import sys
import cv2
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from random import randrange
from tqdm import tqdm, tnrange
from tqdm.notebook import tqdm_notebook

import warnings

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

from keras.optimizer_v2.gradient_descent import SGD
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Dense, MaxPooling2D, Flatten, Conv2D, Lambda, Dropout, LeakyReLU, BatchNormalization, Activation, AveragePooling2D, GlobalAveragePooling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

from mlxtend.evaluate import accuracy

from tensorflow.keras.optimizers import Adam,RMSprop,SGD, Nadam
#from tensorflow.keras.utils import to_categorical
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from tensorflow.keras.applications import VGG16

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # ignore tensorflow warnings
warnings.filterwarnings("ignore")

In [3]:
path_data = "data/data_palm_vein/NIR"

In [4]:
def load_img(path, xdim=128, ydim=128, nb_of_users = 500):
    label_names = []
    X = []
    y = []
    nb_of_users = nb_of_users - 1
    count = 0
    identity = -1
    directories = os.listdir(path)
    directories.sort()
    for dirname in tqdm_notebook(directories, desc="Loading images..."):
        if dirname == ".DS_Store": continue
        label_names.append(dirname)
        data_path = os.path.join(path + "/" + dirname, '*g')
        files = glob.glob(data_path)
        if identity >= nb_of_users: break
        identity += 1
        files.sort()
        for f1 in files:
            img = cv2.imread(f1, cv2.IMREAD_GRAYSCALE)
            #img = cv2.imread(f1)
            #img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
            img = cv2.resize(img,(int(xdim*1), int(ydim*1)))
            X.append(np.array(img))
            y.append(identity)
            count += 1
    X = np.array(X)
    y = np.array(y)
    print("\n ================= Summary of extraction ================= \n")
    print(count, ' images lues')
    print("\nX.shape = ", X.shape)
    print("y.shape = ", y.shape)
    gc.collect()
    return X, y, label_names

In [5]:
X, y, label_names = load_img(path_data, nb_of_users=50, xdim=224, ydim=224)

Loading images...:   0%|          | 0/501 [00:00<?, ?it/s]


 ================= Summary of extraction ================= 

600  images lues

X.shape =  (600, 224, 224)
y.shape =  (600,)


In [6]:
X = X / 255.
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
y = to_categorical(y)

print("\nPREPROCESSING DATA")
print("-----------------------------------------")
print("\nX shape : {}".format(X.shape))
print("-----------------------------------------")
print("y shape : {}\n".format(y.shape))
print("\n\n-----------------------------------------")
print("Il y a {} utilisateur(s) dans le dataset prélevé.".format(y.shape[1]))


PREPROCESSING DATA
-----------------------------------------

X shape : (600, 224, 224, 1)
-----------------------------------------
y shape : (600, 50)



-----------------------------------------
Il y a 50 utilisateur(s) dans le dataset prélevé.


In [18]:

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
print("\nSplitting data ...\n")
print("\n=============================== Splitting data =============================== \n")
print("\nX_train shape : {}    |   y_train shape : {}".format(X_train.shape, y_train.shape))
print("------------------------------------------------------------------------------")
print("(X_temp shape : {}    |   y_temp shape : {})\n".format(X_temp.shape, y_temp.shape))



Splitting data ...


=============================== Splitting data =============================== 


X_train shape : (480, 224, 224, 1)    |   y_train shape : (480, 50)
------------------------------------------------------------------------------
(X_temp shape : (120, 224, 224, 1)    |   y_temp shape : (120, 50))



In [27]:
def vgg_model(input_shape=(128, 128, 1), nombre_classes=500):
    model = Sequential()
    model.add(Conv2D(64, (3, 3), kernel_initializer='he_uniform', activation='LeakyReLU', input_shape=input_shape,
                     padding='same'))
    model.add(Conv2D(64, kernel_size=(3, 3), strides=(2, 2), kernel_initializer='he_uniform', activation='LeakyReLU',
                     padding='same'))
    model.add(MaxPooling2D(strides=(2, 2)))
    model.add(
        Conv2D(128, kernel_size=(3, 3), activation='LeakyReLU', padding='same', kernel_initializer='he_uniform', ))
    model.add(
        Conv2D(128, kernel_size=(3, 3), activation='LeakyReLU', padding='same', kernel_initializer='he_uniform', ))
    model.add(MaxPooling2D(strides=(2, 2)))
    model.add(
        Conv2D(256, kernel_size=(3, 3), activation='LeakyReLU', padding='same', kernel_initializer='he_uniform', ))
    model.add(
        Conv2D(256, kernel_size=(3, 3), activation='LeakyReLU', padding='same', kernel_initializer='he_uniform', ))
    model.add(
        Conv2D(256, kernel_size=(3, 3), activation='LeakyReLU', padding='same', kernel_initializer='he_uniform', ))
    model.add(MaxPooling2D(strides=(2, 2)))
    model.add(
        Conv2D(512, kernel_size=(3, 3), activation='LeakyReLU', padding='same', kernel_initializer='he_uniform', ))
    model.add(
        Conv2D(512, kernel_size=(3, 3), activation='LeakyReLU', padding='same', kernel_initializer='he_uniform', ))
    model.add(
        Conv2D(512, kernel_size=(3, 3), activation='LeakyReLU', padding='same', kernel_initializer='he_uniform', ))
    model.add(MaxPooling2D(strides=(2, 2)))
    model.add(Flatten())
    model.add(Dense(4096, activation='LeakyReLU'))
    model.add(Dense(4096, activation='LeakyReLU'))
    model.add(Dense(nombre_classes, activation='softmax'))

    print(model.summary())
    print("\n ================= VGG16 model ================= \n")

    model.compile(optimizer="nadam", loss='categorical_crossentropy', metrics='accuracy')
    return model

In [28]:
Nombre_classes = y.shape[1]
input_shape = (X.shape[1], X.shape[2], 1)
print("\nNombres de classes : {}   |   Input shape : {}\n".format(Nombre_classes, input_shape))


Nombres de classes : 50   |   Input shape : (224, 224, 1)



In [29]:
model_zfnet = vgg_model(input_shape, Nombre_classes)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 224, 224, 64)      640       
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 112, 112, 64)      36928     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 56, 56, 128)       73856     
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 56, 56, 128)       147584    
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 28, 28, 256)      

In [30]:
epochs = 80
batch_size = 50
print("\n ================= Training : ZFNET model ================= \n")
print("  Epochs :  {}   |   Batch size : {} ".format(epochs, batch_size))
print("\n ========================================================== \n")

trained_model = model_zfnet.fit(X_train, y_train, validation_data=(X_temp, y_temp), epochs=epochs)


 ================= Training : ZFNET model ================= 

  Epochs :  80   |   Batch size : 50 


Epoch 1/80
15/15 [==============================] - 19s 1s/step - loss: 4783.2275 - accuracy: 0.0188 - val_loss: 185.8005 - val_accuracy: 0.0333
Epoch 2/80
15/15 [==============================] - 18s 1s/step - loss: 83.6102 - accuracy: 0.0063 - val_loss: 12.3980 - val_accuracy: 0.0417
Epoch 3/80
15/15 [==============================] - 18s 1s/step - loss: 10.0592 - accuracy: 0.0083 - val_loss: 5.8813 - val_accuracy: 0.0250
Epoch 4/80
15/15 [==============================] - 18s 1s/step - loss: 4.8733 - accuracy: 0.0167 - val_loss: 4.2622 - val_accuracy: 0.0083
Epoch 5/80
15/15 [==============================] - 18s 1s/step - loss: 4.0831 - accuracy: 0.0292 - val_loss: 4.2230 - val_accuracy: 0.0083
Epoch 6/80
15/15 [==============================] - 18s 1s/step - loss: 4.1786 - accuracy: 0.0208 - val_loss: 4.1317 - val_accuracy: 0.0083
Epoch 7/80
15/15 [==============================]

KeyboardInterrupt: 

In [ ]:
val = model_zfnet.evaluate(X_temp, y_temp, verbose=0)

print("\n ================= Evaluation : ZFNET model ================= \n")
print("  With : \n")
print("Batch size         :  {}    |   Epochs      : {} ".format(batch_size, epochs))
print("Nombres de classes :  {}    |   Input shape : {} \n".format(Nombre_classes, input_shape))
print("--------------------------------------------------------------")
print("  Results : \n")
print("Loss  : %.2f%%" % (val[0] * 100))
print("Score : %.2f%%" % (val[1] * 100))

In [ ]:
epoch = 80
batch_size = 50

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=5,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.05, # Randomly zoom image 
        width_shift_range=0,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images
datagen.fit(X_train)

history = model_zfnet.fit_generator(
                              datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epoch, 
                              validation_data = (X_temp,y_temp),
                              steps_per_epoch=X_train.shape[0] // batch_size
                             )

In [ ]:
model_zfnet.evaluate(X_temp, y_temp)